In [12]:
import torch
from PIL import Image
from torch.utils.data import Dataset, ConcatDataset
import torchvision.datasets as datasets
import os
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.optim as optim
import torchvision.models as models
import torch.nn as nn
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch.nn.functional as F
from sklearn.metrics import classification_report
from sklearn import metrics
num_classes = 2 # 1 in this case means binary classification
bsz = 64

#model_dir = '/home/charlietran/nsf_sch/model_weights/Experiment_AD_CN_Matched_Corrected/testfold_1_v2.pth'
#test_img_dir = '/DATA/charlie/fundus/Redo/Rematching/Experiment_AD_CN_Corrected/f1/'

model_dir = r'C:\Users\Tran\Dropbox (UFL)\NSF_SCH\code\AutoMorph_6180\model\Experiment\VGG16_AutoMorph_TESTFOLD_5_6180_1300.pth'
test_img_dir = r'C:\Users\Tran\Dropbox (UFL)\NSF_SCH\data\AutoMorph_6180\Experiment\AutoMorph_6180_1300\f5'

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225])


test_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    normalize,
])


test_fold = datasets.ImageFolder(test_img_dir, transform = test_transform)
test_dataloader = DataLoader(test_fold, batch_size = bsz, shuffle = False)

model = models.vgg16(pretrained=True)
model.classifier[6] = nn.Linear(in_features=4096, out_features= 2, bias=True)
checkpoint = torch.load(model_dir)
model.load_state_dict(checkpoint['model_state_dict'])
#model = torch.nn.DataParallel(model, device_ids=[0,1])
model.cuda()


def computeAUC(dataGT, dataPRED):
    outAUROC = []
    datanpGT = dataGT.cpu().numpy()
    datanpPRED = dataPRED.cpu().numpy()
            
    return roc_auc_score(datanpGT, datanpPRED)

def classreport(dataGT, predCLASS):
    datanpGT = dataGT.cpu().numpy()
    datanppredCLASS = predCLASS.cpu().numpy()
    print(classification_report(datanpGT, datanppredCLASS, digits = 3))
    
def computeACC(dataGT, dataCLASS):
    
    datanpGT = dataGT.cpu().numpy()
    datanpCLASS = dataCLASS.cpu().numpy()
    
    return accuracy_score(datanpGT, datanpCLASS)

def test(model, dataloader):
    model.eval()
    model.cuda()
    outGT = torch.FloatTensor().cuda()
    outPRED = torch.FloatTensor().cuda()
    outClass = torch.FloatTensor().cuda()
    with torch.no_grad():
        for i, data in enumerate(dataloader):
            images, labels, = data
            images = images.float().cuda()
            labels = labels.float().cuda()
            output = model(images)
            #outPRED = torch.cat((outPRED, torch.sigmoid(output)), 0)
            
            outGT = torch.cat((outGT, labels), 0)
            #softmax_preds = torch.sigmoid(output)
            softmax_preds = F.softmax(output, dim = 1)
            #outPRED = torch.cat((outPRED, torch.sigmoid(output)), 0)
            outPRED = torch.cat((outPRED, softmax_preds), 0)
            
            #class_predictions = torch.round(softmax_preds)
            #class_predictions = torch.argmax(softmax_preds, dim = 1)
            class_predictions = torch.argmax(softmax_preds, dim = 1)
            outClass = torch.cat((outClass, class_predictions), 0)
            
    classreport(outGT, outClass)
    #print(outPRED, outPRED.shape)
    #auc_test = computeAUC(outGT, outPRED)
    
    acc_test = computeACC(outGT, outClass)
    #return auc_test, accuracy_test
    return acc_test


acc_test = test(model, test_dataloader)


              precision    recall  f1-score   support

         0.0      0.765     0.650     0.703        20
         1.0      0.696     0.800     0.744        20

    accuracy                          0.725        40
   macro avg      0.730     0.725     0.723        40
weighted avg      0.730     0.725     0.723        40

